In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/STAT_440/nft")

In [ ]:
!ls

data_current  models.ipynb  preprocessing.ipynb


In [ ]:
!unzip "/content/gdrive/My Drive/STAT_440/nft/stat440-21-project2.zip" -d "./data_images"
!ls

Streaming output truncated to the last 5000 lines.
  inflating: ./data_images/images/images/6f45c0de111e463217778b6b0eafb23e.jpg  
  inflating: ./data_images/images/images/6f464162a1f0516bc14963613bbb58c2.png  
  inflating: ./data_images/images/images/6f5e3533ff98a46eadc4703108d37570.jpg  
  inflating: ./data_images/images/images/6f5ef11b8fade64710be35e226b3045f.png  
  inflating: ./data_images/images/images/6f65432d7e85112bbffb37577a875ed1.png  
  inflating: ./data_images/images/images/6f67899280ca22aa4e70fccb46b699e9.png  
  inflating: ./data_images/images/images/6f6f1e3070b5596922dc28173b7d2555.jpg  
  inflating: ./data_images/images/images/6f71fd897629ea6e68eb8203cbeb651b.png  
  inflating: ./data_images/images/images/6f75c61fff22a6bde0ed2db59586f72b.png  
  inflating: ./data_images/images/images/6f78b233eec57994453e09817351ec2f.gif  
  inflating: ./data_images/images/images/6f7aaf3732d3e8b4da5c9ca32a4cdfa6.png  
  inflating: ./data_images/images/images/6f8d7ceda6575e5175b98f5ac6cd

In [ ]:
!pip3 install numpy pandas scikit-learn seaborn matplotlib xgboost Pillow scikit-image feature-engine scipy tqdm

     |████████████████████████████████| 180 kB 5.0 MB/s 
     |████████████████████████████████| 9.8 MB 41.3 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# Text processing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
# Image processing
from skimage.io import imread, imshow
from skimage.color import rgba2rgb
from skimage.transform import resize
from skimage.filters import sobel
# Other processing
from sklearn.preprocessing import StandardScaler
from feature_engine.creation import CyclicalTransformer
from sklearn.decomposition import TruncatedSVD

In [162]:
# Load CSV to Dataframe
PATH = './data_current/'
TRAIN_FILE = 'XYtr.csv'
TEST_FILE = 'Xte.csv'
IMAGE_FOLDER = PATH+'../data_images/images/images/'
df_train = pd.read_csv(PATH + TRAIN_FILE, index_col='id')
df_test = pd.read_csv(PATH + TEST_FILE, index_col='id')

# So description, version, symbol, fee1, and fee2 have missing values (NaN)
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6914 entries, a44a5f4c5e13910205404271e750e7bc to 62defe67d57479ab0cd6d1ffb6525cbb
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   X.sales      6914 non-null   int64  
 1   cdate        6914 non-null   object 
 2   description  6512 non-null   object 
 3   version      6746 non-null   object 
 4   symbol       5555 non-null   object 
 5   ext          6914 non-null   object 
 6   fee1         6696 non-null   float64
 7   fee2         6705 non-null   float64
 8   total        6914 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 540.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 6914 entries, 7e79f1a9cb10504dd2fc569d84f2a346 to 3b665129694904b2024dc7cd8230babe
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   X.sales      6914 non-null   int64  
 1   cdate        6914 

In [163]:
'''
Data Cleaning
'''

# description: use the token None to mean no description
df_train['description'] = df_train['description'].fillna('None')
df_test['description'] = df_test['description'].fillna('None')

# version: Has 'None' category. Set nan to 'None'. 
df_train['version'] = df_train['version'].fillna('None')
df_test['version'] = df_test['version'].fillna('None')

# symbol: 5 digit symbols. Set to 00000 to represent None.
# print(df_test['symbol'].unique())
df_train['symbol'] = df_train['symbol'].fillna('00000')
df_test['symbol'] = df_test['symbol'].fillna('00000')

# fee1: Small number misssin. Fill with the mean.
df_train['fee1'] = df_train['fee1'].fillna((df_train['fee1'].median()))
df_test['fee1'] = df_test['fee1'].fillna((df_test['fee1'].median()))

# fee2: Small number misssin. Fill with the mean.
df_train['fee2'] = df_train['fee2'].fillna((df_train['fee2'].median()))
df_test['fee2'] = df_test['fee2'].fillna((df_test['fee2'].median()))

print(df_train.info())
print(df_test.info())

# Remove price outliers
df_train = df_train[np.abs(df_train['total']-df_train['total'].mean()) <= (4*df_train['total'].std())]
print(df_train.info())


<class 'pandas.core.frame.DataFrame'>
Index: 6914 entries, a44a5f4c5e13910205404271e750e7bc to 62defe67d57479ab0cd6d1ffb6525cbb
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   X.sales      6914 non-null   int64  
 1   cdate        6914 non-null   object 
 2   description  6914 non-null   object 
 3   version      6914 non-null   object 
 4   symbol       6914 non-null   object 
 5   ext          6914 non-null   object 
 6   fee1         6914 non-null   float64
 7   fee2         6914 non-null   float64
 8   total        6914 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 540.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 6914 entries, 7e79f1a9cb10504dd2fc569d84f2a346 to 3b665129694904b2024dc7cd8230babe
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   X.sales      6914 non-null   int64  
 1   cdate        6914 

In [164]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]], drop_first=True)
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

def dim_reduction(df, df_train_test, K, column_name=None):
  cols = [column_name + '_' + str(col) for col in range(K)]
  svd = TruncatedSVD(n_components=K, n_iter=7, random_state=42)
  res = svd.fit_transform(df)
  reduced_df = pd.DataFrame(res, columns=cols, index=df_train_test.index)
  print('Total variance explained for K={} is {}'.format(K,svd.explained_variance_ratio_.sum()))

  return df_train_test.join(reduced_df, how='left')  


# We need to combine the train and test sets so we can get the same number of features on each.
train_size = len(df_train)
test_size = len(df_test)

df_train_test =  df_train.append(df_test)

# One-hot encode version
df_train_test = encode_and_bind(df_train_test, 'version')

# One-hot encode symbol
df_train_test = encode_and_bind(df_train_test, 'symbol')
# Perform dimension reduction
symbol_features = df_train_test.columns.tolist()
symbol_features = [feat for feat in symbol_features if feat.startswith('symbol')]
symbols_df = df_train_test[symbol_features]
df_train_test = df_train_test.drop(columns=symbol_features)
df_train_test = dim_reduction(symbols_df, df_train_test, 30, column_name='symbol')
print(df_train_test.columns.tolist())
print(df_train_test.shape)

# One-hot encode ext
df_train_test = encode_and_bind(df_train_test, 'ext')

Total variance explained for K=30 is 0.7872245148675877
['X.sales', 'cdate', 'description', 'ext', 'fee1', 'fee2', 'total', 'version_2', 'version_3', 'version_4', 'version_None', 'version_unsupported', 'symbol_0', 'symbol_1', 'symbol_2', 'symbol_3', 'symbol_4', 'symbol_5', 'symbol_6', 'symbol_7', 'symbol_8', 'symbol_9', 'symbol_10', 'symbol_11', 'symbol_12', 'symbol_13', 'symbol_14', 'symbol_15', 'symbol_16', 'symbol_17', 'symbol_18', 'symbol_19', 'symbol_20', 'symbol_21', 'symbol_22', 'symbol_23', 'symbol_24', 'symbol_25', 'symbol_26', 'symbol_27', 'symbol_28', 'symbol_29']
(13756, 42)


In [165]:
def extract_text_features(method, df_train_test):
    
    # Create the corpus using the training and test data
    description = df_train_test['description']
    corpus = list(description)
    vectorizer = CountVectorizer()
    corpus = vectorizer.fit_transform(corpus)
    features = vectorizer.get_feature_names()
    
    # Create a simple bag of words
    if method == 'BOG':
        sequences = list(description)
        sequences = vectorizer.transform(sequences).toarray()
        word_features = pd.DataFrame(sequences, columns=features, index=df_train_test.index)

    # Perform dim reduction on text processing
    df_train_test = dim_reduction(word_features, df_train_test, 50, 'desc')
    return df_train_test


df_train_test = extract_text_features('BOG', df_train_test)
print(df_train_test.columns.tolist())

Total variance explained for K=50 is 0.7426068425652803
['X.sales', 'cdate', 'description', 'fee1', 'fee2', 'total', 'version_2', 'version_3', 'version_4', 'version_None', 'version_unsupported', 'symbol_0', 'symbol_1', 'symbol_2', 'symbol_3', 'symbol_4', 'symbol_5', 'symbol_6', 'symbol_7', 'symbol_8', 'symbol_9', 'symbol_10', 'symbol_11', 'symbol_12', 'symbol_13', 'symbol_14', 'symbol_15', 'symbol_16', 'symbol_17', 'symbol_18', 'symbol_19', 'symbol_20', 'symbol_21', 'symbol_22', 'symbol_23', 'symbol_24', 'symbol_25', 'symbol_26', 'symbol_27', 'symbol_28', 'symbol_29', 'ext_.jpg', 'ext_.png', 'desc_0', 'desc_1', 'desc_2', 'desc_3', 'desc_4', 'desc_5', 'desc_6', 'desc_7', 'desc_8', 'desc_9', 'desc_10', 'desc_11', 'desc_12', 'desc_13', 'desc_14', 'desc_15', 'desc_16', 'desc_17', 'desc_18', 'desc_19', 'desc_20', 'desc_21', 'desc_22', 'desc_23', 'desc_24', 'desc_25', 'desc_26', 'desc_27', 'desc_28', 'desc_29', 'desc_30', 'desc_31', 'desc_32', 'desc_33', 'desc_34', 'desc_35', 'desc_36', 'des

In [166]:
def image_text_features(method, df_train_test):
    
    '''
    Setup image features.
    '''
    # Get cached image dataframe if available
    IMAGE_FILE_NAME = 'cached_image_{}.csv'.format(method)
    files = os.listdir(PATH)
    # if IMAGE_FILE_NAME in files:
    #     return pd.read_csv(PATH + IMAGE_FILE_NAME, index_col='id')

    IMAGE_HEIGHT = 16
    IMAGE_WIDTH = 16
    indexes = list(range(len(df_train_test)))
    image_features = ['pixel{}'.format(num) for num in range(IMAGE_HEIGHT*IMAGE_WIDTH)]
    image_features_df = pd.DataFrame(0, index=indexes, columns=image_features)
    image_features_df.index = df_train_test.index
    
    images = os.listdir(IMAGE_FOLDER)
    for i in tqdm(range(len(images))):
        image_file = images[i]
        image_id = image_file.split('.')[0]
        if image_id not in indexes: continue

        if method == 'EDGE':
            image = imread('{}/{}'.format(IMAGE_FOLDER, image_file), as_gray=True)
            height, width = image.shape
            if height*width > 96*96: continue
            image = resize(image, (IMAGE_HEIGHT, IMAGE_WIDTH))

            edge_sobel = sobel(image)
            edge_sobel = edge_sobel.reshape((-1, IMAGE_HEIGHT*IMAGE_WIDTH))
            image_features_df.loc[image_id,:] = edge_sobel.tolist()[0]

    df_train_test = dim_reduction(image_features_df, df_train_test, 20, column_name='pixel')    
    df_train_test.to_csv(PATH + IMAGE_FILE_NAME)
    return df_train_test


print(df_train_test.shape)
df_train_test = image_text_features('EDGE', df_train_test)
print(df_train_test.shape)


(13756, 93)


100%|██████████| 8873/8873 [00:02<00:00, 3208.68it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


Total variance explained for K=20 is nan
(13756, 113)


In [167]:
df_dates = df_train_test[['cdate']].copy()
df_dates['cdate'] = df_dates['cdate'].astype('datetime64[ns]')
df_dates['year'] = pd.DatetimeIndex(df_dates['cdate']).year
df_dates['month'] = pd.DatetimeIndex(df_dates['cdate']).month
df_dates['day'] = pd.DatetimeIndex(df_dates['cdate']).day

cyclical = CyclicalTransformer(variables=None, drop_original=True)
df_dates_new = cyclical.fit_transform(df_dates)

df_dates_new = df_dates_new.drop(columns=['cdate'])
df_train_test = df_train_test.drop(columns=['cdate'])
df_train_test = df_train_test.join(df_dates_new, how='left')


In [168]:
'''
Drop remaining unecessary data. Seperate data
'''
train_features = df_train_test.iloc[:train_size]
test_features = df_train_test.iloc[train_size:]

train_features = train_features.drop(columns=['description'])
test_features = test_features.drop(columns=['total','description'])


In [169]:
train_features.to_csv('data_current/XYtr_updated_reduced_dim1.csv')
test_features.to_csv('data_current/Xte_updated_reduced_dim1.csv')